In [ ]:
 !find train/* -type f | wc -l

In [ ]:
 !find test/* -type f | wc -l

In [ ]:
!mv `ls train/* | shuf -n 1022` dev/

In [ ]:
 !find dev/* -type f | wc -l

In [ ]:
 !find train/* -type f | wc -l

In [ ]:
import shutil
import os
import pandas as pd
from tqdm import tqdm

In [ ]:
df = pd.read_csv('labels.csv')

In [ ]:
path = 'data/categorical/train'

if os.path.exists(path):
    print('path exists')
    shutil.rmtree(path)
    
for i, (fname, breed) in tqdm(df.iterrows()):
    path2 = '%s/%s' % (path, breed)
    if not os.path.exists(path2):
        os.makedirs(path2)
    shutil.copyfile('data/original/train/%s.jpg' % fname, '%s/%s.jpg' % (path2, fname))

In [ ]:
df = pd.read_csv('sample_submission.csv')
path = 'data/categorical/test'
breed = 'unknown'

if os.path.exists(path):
    shutil.rmtree(path)

for fname in tqdm(df['id']):
    path2 = '%s/%s' % (path, breed)
    if not os.path.exists(path2):
        os.makedirs(path2)
    shutil.copyfile('data/original/test/%s.jpg' % fname, '%s/%s.jpg' % (path2, fname))

#### Import dependencies

In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.02
tf.Session(config=config)
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import csv
import os
from PIL import Image
from tqdm import tqdm
%matplotlib inline

In [ ]:
datagen = ImageDataGenerator(
    rotation_range = 40,        
    horizontal_flip = True,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    rescale = 1./255,
    shear_range=0.2,    
    zoom_range = 0.2,
    fill_mode='nearest')


In [ ]:
img = load_img('train/bf4cee97c84ad782f0bcb59b9d42e022.jpg')

In [ ]:
print(type(img))
img

In [ ]:
x = img_to_array(img)
print(type(x))
print(x.shape)

In [ ]:
x = x.reshape((1, ) + x.shape) # 4 channels are required for image data generator in keras

In [ ]:
print(img)

In [ ]:
i = 0
for batch in datagen.flow(x, batch_size=1, save_to_dir='aug', save_prefix='lulz', save_format='jpeg'):
    i+=1
    if i>20:break

## preprocess

In [ ]:
df_train = pd.read_csv('labels.csv')

In [ ]:
df_test = pd.read_csv('sample_submission.csv')

In [ ]:
print(df_train.shape)
df_train.head(3)

In [ ]:
print(df_test.shape)
df_test.head(3)

In [ ]:
# targets_series = pd.Series(train_label['breed'])
# print(targets_series.shape)
# targets_series.head()
df_train['breed'].shape

In [ ]:
one_hot = pd.get_dummies(df_train['breed'], sparse = True)

In [ ]:
one_hot.head(3)

In [ ]:
one_hot_labels = np.asarray(one_hot)

In [ ]:
print(one_hot_labels.shape)
one_hot_labels

In [ ]:
X_test = []

In [ ]:
X_train = []
Y_train = []
X_test = []

In [ ]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = img_to_array(load_img('data/train/{}.jpg'.format(f), target_size=(224, 224, 3)))
    X_train.append(img)
    Y_train.append(one_hot_labels[i])
    i += 1

In [ ]:
for f in tqdm(df_test['id'].values):
    img = img_to_array(load_img('data/test/{}.jpg'.format(f), target_size=(224, 224, 3)))
    X_test.append(img)

In [ ]:
X_train = np.array(X_train, np.float32) / 255.
Y_train = np.array(Y_train, np.uint8)
X_test  = np.array(X_test, np.float32) / 255.

In [ ]:
X_train.shape, Y_train.shape, X_test.shape

In [ ]:
np.save('data/processed/X_train.npy', X_train)
np.save('data/processed/Y_train.npy', Y_train)
np.save('data/processed/X_test.npy', X_test)


### old stuff

In [ ]:
# train_label, dev_label = train_test_split(label_data, test_size=0.10, random_state=0, stratify=label_data['breed'])

In [ ]:
label_data.loc[label_data['id']=='00214f311d5d2247d5dfe4fe24b2303d']['breed'].as_matrix()[0]

In [ ]:
unique_labels = pd.unique(label_data['breed'])
len(unique_labels), unique_labels[105]
ul = list(unique_labels)

In [ ]:
ul.index('pomeranian')

In [ ]:
# labels = pd.value_counts(label_data['breed'])#to get index of any element in numpy array
np.where(unique_labels=='pomeranian')[0][0]

In [ ]:
# X as float32 won't show exact color image, turn its dtype to uint8 to see how it looks actually, it's nessessary to set X to float32
def load_data(label_data):
    X = np.zeros(([label_data.shape[0], 224, 224, 3]), dtype='float32')
    Y = np.zeros(([label_data.shape[0], 120]), dtype='float32')
    for i, id_ in enumerate(label_data['id']):
        file = 'train/' + id_ + '.jpg'
        img = mpimg.imread(file)
        X[i] = img
        breed = label_data.loc[label_data['id']==id_]['breed'].as_matrix()[0]
        Y[i][ul.index(breed)] = 1.
    return (X, Y)
